# ADVANCED SQL 2:  Advanced Joins and Performance Tuning

We connect to MySQL server and workbench and make analysis with the parch-and-posey database. This course is the practicals of the course SQL for Data Analysis at Udacity.

In [1]:
# we import some required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import time
print('Done!')

Done!


In [2]:
import mysql
from mysql.connector import Error
from getpass import getpass

db_name = 'parch_and_posey'
try:
    connection = mysql.connector.connect(host='localhost',
                                         database=db_name,
                                         user=input('Enter UserName:'),
                                         password=getpass('Enter Password:'))
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)

Enter UserName:danam
Enter Password:········
Connected to MySQL Server version  8.0.24
You're connected to database:  ('parch_and_posey',)


In [3]:
def query_to_df(query):
    st = time.time()
    # Assert Every Query ends with a semi-colon
    try:
        assert query.endswith(';')
    except AssertionError:
        return 'ERROR: Query Must End with ;'

    # so we never have more than 20 rows displayed
    pd.set_option('display.max_rows', 30) 
    df = None

    # Process the query
    cursor.execute(query)
    columns = cursor.description
    result = []
    for value in cursor.fetchall():
        tmp = {}
        for (index,column) in enumerate(value):
            tmp[columns[index][0]] = [column]
        result.append(tmp)

    # Create a DataFrame from all results
    for ind, data in enumerate(result):
        if ind >= 1:
            x = pd.DataFrame(data)
            df = pd.concat([df, x], ignore_index=True)
        else:
            df = pd.DataFrame(data)
    print(f'Query ran for {time.time()-st} secs!')
    return df

In [4]:
# Let's see the tables in Parch-and-Posey database

query_to_df(
'SHOW TABLES;'
)

Query ran for 0.0096435546875 secs!


,Tables_in_parch_and_posey
0,accounts
1,orders
2,region
3,sales_reps
4,web_events


In [5]:
# 1. For the accounts table

query = 'SELECT * FROM accounts LIMIT 3;'
query_to_df(query)

Query ran for 0.012177705764770508 secs!


,id,name,website,lat,longs,primary_poc,sales_rep_id
0,1001,Walmart,www.walmart.com,40.23849561,-75.10329704,Tamara Tuma,321500
1,1011,Exxon Mobil,www.exxonmobil.com,41.16915630,-73.84937379,Sung Shields,321510
2,1021,Apple,www.apple.com,42.29049481,-76.08400942,Jodee Lupo,321520


In [6]:
# 2. For the orders table

query = 'SELECT * FROM orders LIMIT 3;'
query_to_df(query)

Query ran for 0.01546168327331543 secs!


,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18


In [7]:
# 3. For the sales_reps table

query = 'SELECT * FROM sales_reps LIMIT 3;'
query_to_df(query)

Query ran for 0.008377313613891602 secs!


,id,name,region_id
0,321500,Samuel Racine,1
1,321510,Eugena Esser,1
2,321520,Michel Averette,1


In [8]:
# 4. For the web_events table

query = 'SELECT * FROM web_events LIMIT 3;'
query_to_df(query)

Query ran for 0.011348485946655273 secs!


,id,account_id,occurred_at,channel
0,1,1001,2015-10-06 17:13:58,direct
1,2,1001,2015-11-05 03:08:26,direct
2,3,1001,2015-12-04 03:57:24,direct


In [9]:
# 5. For the region table

query = 'SELECT * FROM region LIMIT 3;'
query_to_df(query)

Query ran for 0.0062732696533203125 secs!


,id,name
0,1,Northeast
1,2,Midwest
2,3,Southeast


## 1. Inner Join:

<img src='https://video.udacity-data.com/topher/2017/November/5a147488_inner-join/inner-join.png' height=200 width=300>

**Inner Joins** produce results for which the join condition is matched in both tables. It is the default kind of join. Frankly, we can perform the Inner-join with just the key-word Join. SQL automatically does Inner-join on the Join clause.

In [10]:
query_to_df(
"SELECT a.*, s.* FROM accounts a INNER JOIN sales_reps s ON a.sales_rep_id=s.id;"
)

Query ran for 1.3931076526641846 secs!


,id,name,website,lat,longs,primary_poc,sales_rep_id,region_id
0,321500,Samuel Racine,www.walmart.com,40.23849561,-75.10329704,Tamara Tuma,321500,1
1,321510,Eugena Esser,www.exxonmobil.com,41.16915630,-73.84937379,Sung Shields,321510,1
2,321520,Michel Averette,www.apple.com,42.29049481,-76.08400942,Jodee Lupo,321520,1
3,321530,Renetta Carew,www.berkshirehathaway.com,40.94902131,-75.76389759,Serafina Banda,321530,1
4,321540,Cara Clarke,www.mckesson.com,42.21709326,-75.28499823,Angeles Crusoe,321540,1
...,...,...,...,...,...,...,...,...
346,321970,Georgianna Chisholm,www.kkr.com,45.54535285,-122.65524711,Buffy Azure,321970,4
347,321960,Maryanna Fiorentino,www.oneok.com,45.51351286,-122.68150011,Esta Engelhardt,321960,4
348,321970,Georgianna Chisholm,www.newmont.com,45.49411669,-122.66946042,Khadijah Riemann,321970,4
349,321960,Maryanna Fiorentino,www.pplweb.com,45.49172048,-122.67187978,Deanne Hertlein,321960,4


## 2. Left Join:

<img src='https://video.udacity-data.com/topher/2017/November/5a147484_left-join/left-join.png' height=200 width=300>

**Left Joins** also include unmatched rows from the left table, which is indicated in the “FROM” clause.<br>**NOTE That:** Left-Join and Left-Outer-Join are the same

In [11]:
query_to_df(
"SELECT a.*, s.* FROM accounts a LEFT JOIN sales_reps s ON a.sales_rep_id=s.id;"
)

Query ran for 1.3927791118621826 secs!


,id,name,website,lat,longs,primary_poc,sales_rep_id,region_id
0,321500,Samuel Racine,www.walmart.com,40.23849561,-75.10329704,Tamara Tuma,321500,1
1,321510,Eugena Esser,www.exxonmobil.com,41.16915630,-73.84937379,Sung Shields,321510,1
2,321520,Michel Averette,www.apple.com,42.29049481,-76.08400942,Jodee Lupo,321520,1
3,321530,Renetta Carew,www.berkshirehathaway.com,40.94902131,-75.76389759,Serafina Banda,321530,1
4,321540,Cara Clarke,www.mckesson.com,42.21709326,-75.28499823,Angeles Crusoe,321540,1
...,...,...,...,...,...,...,...,...
346,321970,Georgianna Chisholm,www.kkr.com,45.54535285,-122.65524711,Buffy Azure,321970,4
347,321960,Maryanna Fiorentino,www.oneok.com,45.51351286,-122.68150011,Esta Engelhardt,321960,4
348,321970,Georgianna Chisholm,www.newmont.com,45.49411669,-122.66946042,Khadijah Riemann,321970,4
349,321960,Maryanna Fiorentino,www.pplweb.com,45.49172048,-122.67187978,Deanne Hertlein,321960,4


## 3. Right Join:

<img src='https://video.udacity-data.com/topher/2017/November/5a147485_right-join/right-join.png' height=200 width=300>

**Right Joins** are similar to left joins, but include unmatched data from the right table -- the one that’s indicated in the JOIN clause..<br>**NOTE That:** Right-Join and Right-Outer-Join are the same

In [12]:
query_to_df(
"SELECT a.*, s.* FROM accounts a RIGHT JOIN sales_reps s ON a.sales_rep_id=s.id;"
)

Query ran for 1.3683652877807617 secs!


,id,name,website,lat,longs,primary_poc,sales_rep_id,region_id
0,321500,Samuel Racine,www.johnsoncontrols.com,40.76346837,-73.97890353,Cammy Sosnowski,321500,1
1,321500,Samuel Racine,www.aa.com,40.77158500,-73.98195470,Sasha Haughey,321500,1
2,321500,Samuel Racine,www.ingrammicro.com,40.76945841,-73.98081794,Chanelle Keach,321500,1
3,321500,Samuel Racine,www.freddiemac.com,42.77051343,-75.18810094,Elayne Grunewald,321500,1
4,321500,Samuel Racine,www.express-scripts.com,41.47633009,-76.07993254,Jewell Likes,321500,1
...,...,...,...,...,...,...,...,...
346,321990,Dawna Agnew,www.ebay.com,34.04517853,-118.24477160,Monet Maclaren,321990,4
347,321990,Dawna Agnew,www.uhsinc.com,34.05458082,-118.24857583,Tasha Delillo,321990,4
348,321990,Dawna Agnew,www.autozone.com,34.05826339,-118.23774700,Maryellen Strohmeyer,321990,4
349,321990,Dawna Agnew,www.nscorp.com,34.05292101,-118.24011491,Leia Vargas,321990,4


## 4. Full-Outer Join:

<img src='https://video.udacity-data.com/topher/2017/November/5a147487_full-outer-join/full-outer-join.png' height=200 width=300>

**Full-Outer Joins** In some cases, you might want to include unmatched rows from both tables being joined. You can do this with a full outer join.<br>**NOTE That:** Full-Outer-Join and Full-Join are the same

A common application of this is when joining two tables on a timestamp. Let’s say you’ve got one table containing the number of item 1 sold each day, and another containing the number of item 2 sold. If a certain date, like January 1, 2018, exists in the left table but not the right, while another date, like January 2, 2018, exists in the right table but not the left:

a left join would drop the row with January 2, 2018 from the result set
a right join would drop January 1, 2018 from the result set
The only way to make sure both January 1, 2018 and January 2, 2018 make it into the results is to do a full outer join. A full outer join returns unmatched records in each table with null values for the columns that came from the opposite table.

If you wanted to return unmatched rows only, which is useful for some cases of data assessment, you can isolate them by adding the following line to the end of the query:

WHERE Table_A.column_name IS NULL OR Table_B.column_name IS NULL

#### Unfortunately MySQL doesn't support a Full-Outer-Join directly... No FULL JOIN clause.

Therefore to simulate a Full-Outer-Join, we'd combine a Left-Join and a Right-Join using a UNION ALL clause like so...

```
"SELECT * FROM accounts a LEFT JOIN orders o ON a.id=o.account_id 
UNION ALL 
SELECT * FROM accounts a RIGHT JOIN orders o ON a.id=o.account_id 
WHERE a.id IS NULL;"
```

In [ ]:
query_to_df(
    "SELECT * FROM accounts a LEFT JOIN orders o ON a.id=o.account_id \
    UNION ALL \
    SELECT * FROM accounts a RIGHT JOIN orders o ON a.id=o.account_id \
    WHERE a.id IS NULL;"
)

### Quiz: FULL OUTER JOIN

#### Finding Matched and Unmatched Rows with FULL OUTER JOIN

You’re not likely to use `FULL JOIN (which can also be written as FULL OUTER JOIN)` too often, but the syntax is worth practicing anyway. `LEFT JOIN` and `RIGHT JOIN` each return unmatched rows from one of the tables—`FULL JOIN` returns unmatched rows from both tables. `FULL JOIN` is commonly used in conjunction with aggregations to understand the amount of overlap between two tables.

**Say you're an analyst at Parch & Posey and you want to see:**

* each account who has a sales rep and each sales rep that has an account (all of the columns in these returned rows will be full)
* but also each account that does not have a sales rep and each sales rep that does not have an account (some of the columns in these returned rows will be empty)

This type of question is rare, but `FULL OUTER JOIN` is perfect for it. Write a query with FULL OUTER JOIN to fit the above described Parch & Posey scenario (selecting all of the columns in both of the relevant tables, accounts and sales_reps) then answer the subsequent multiple choice quiz.

In [ ]:
query_to_df(
"SELECT * FROM accounts a LEFT JOIN sales_reps s ON a.sales_rep_id=s.id \
UNION ALL \
SELECT * FROM accounts a RIGHT JOIN sales_reps s ON a.sales_rep_id=s.id \
WHERE a.sales_rep_id IS NULL OR s.id IS NULL;"
)

## Joining without an Equals Sign

#### Inequality JOIN:

* One thing to keep in mind is that it's a little bit harder to predict what the results will look like when joining using inequalities. So the result won't give us any hint as to whether we've got the right query. Thus when doing inequality joins, we must pay rapt attention to the query logic being executed.

**EXPERT TIP:**

If you recall from earlier lessons on joins, the `JOIN` clause is evaluated before the `WHERE` clause -- filtering in the `JOIN` clause will eliminate rows before they are joined, while filtering in the `WHERE` clause will leave those rows in and produce some `NULLs`.

**Inequality JOINs**

Inequality operators (**`a.k.a. comparison operators`**) don't only need to be date times or numbers, they also work on strings

**QUIZ:**

Write a query that left joins the accounts table and the sales_reps tables on each sale rep's ID number and joins it using the < comparison operator on accounts.primary_poc and sales_reps.name. Result should have 3 columns:- The account_name, primary_poc and sales_rep name

In [19]:
query_to_df(
"SELECT a.name acct_name, a.primary_poc primary_poc, s.name sales_rep FROM accounts a LEFT JOIN sales_reps s \
ON a.sales_rep_id=s.id AND a.primary_poc < s.name LIMIT 10;"
)

Query ran for 0.019442081451416016 secs!


,acct_name,primary_poc,sales_rep
0,Walmart,Tamara Tuma,None
1,Exxon Mobil,Sung Shields,None
2,Apple,Jodee Lupo,Michel Averette
3,Berkshire Hathaway,Serafina Banda,None
4,McKesson,Angeles Crusoe,Cara Clarke
5,UnitedHealth Group,Savanna Gayman,None
6,CVS Health,Anabel Haskell,Elba Felder
7,General Motors,Barrie Omeara,Shawanda Selke
8,Ford Motor,Kym Hagerman,Sibyl Lauria
9,AT&T,Jamel Mosqueda,Necole Victory


### SELF JOIN:

Sometimes, we need to join a table with itself. Most of the time, we'd do this when we want to find two events that both occurred one after the other within a given time frame. For example, if we want to know which accounts made multiple orders within 30 days. We can find this by joining the orders table unto itself with an inequality join.

**Expert Tip**

This comes up pretty commonly in **job interviews**. Self JOIN logic can be pretty tricky -- often with multiple conditional statements. It is important to pause and think through each step when joining a table to itself.

**Note**

One of the most common use cases for self JOINs is in cases where two events occurred, one after another. As you may have noticed, using inequalities in conjunction with self JOINs is common.

#### QUIZ:

Use a `SELF JOIN` on the web_events table to find those events that re-occurred not more than one day after their first occurrence. Add a column for the `channel` variable in both instances of the table in your query

In [27]:
query_to_df(
"SELECT w1.account_id 1st_acct_id, w1.occurred_at 1st_occurred, w1.channel 1st_channel, \
w2.account_id 2nd_acct_id, w2.occurred_at 2nd_occurred, w2.channel 2nd_channel FROM web_events w1 \
JOIN web_events w2 ON w1.account_id = w2.account_id AND w2.occurred_at > w1.occurred_at AND \
DATEDIFF(w2.occurred_at, w1.occurred_at) <= 1 ORDER BY 1, 2;"
)

Query ran for 7.970627784729004 secs!


,1st_acct_id,1st_occurred,1st_channel,2nd_acct_id,2nd_occurred,2nd_channel
0,1001,2015-10-06 04:22:11,facebook,1001,2015-10-06 17:13:58,direct
1,1001,2015-10-22 05:02:47,organic,1001,2015-10-22 14:04:20,adwords
2,1001,2015-11-05 03:08:26,direct,1001,2015-11-05 17:18:54,direct
3,1001,2016-01-01 15:45:54,adwords,1001,2016-01-02 00:55:03,direct
4,1001,2016-05-01 14:26:40,direct,1001,2016-05-01 15:26:44,direct
...,...,...,...,...,...,...
1190,4491,2013-12-18 09:56:50,direct,4491,2013-12-18 12:48:40,adwords
1191,4491,2016-01-11 02:26:45,adwords,4491,2016-01-11 10:37:21,banner
1192,4491,2016-04-04 05:37:02,twitter,4491,2016-04-04 15:57:45,facebook
1193,4491,2016-10-04 15:43:29,facebook,4491,2016-10-04 23:42:41,twitter


In [30]:
# Change False to True below and run cell to terminate connection

if True and connection.is_connected():
    cursor.close()
    connection.close()
    print(f'Connection Terminated: {record} Database.')

Connection Terminated: ('parch_and_posey',) Database.
